In [ ]:
import sc2ts
import nb_utils
import numpy as np
from IPython.display import HTML

ts = nb_utils.load()
df = sc2ts.node_data(ts).set_index("sample_id")
arg = nb_utils.D3ARG_viz(ts, df, pangolin_field="pango")
arg.set_sc2ts_node_labels()
arg.set_sc2ts_node_styles()
display(HTML("<style>@media print {.d3arg {zoom: 0.8}}</style>"));

# Export to the `figures` directory (after executing the entire notebook) using 
#   python src/makepdf.py notebooks/Viridian-Pango-XA-XN-XZ.ipynb figures

In [ ]:
# Exclude all XAs apart from a select few
dfXA = df[np.logical_and(df.pango == "XA", df.is_sample)]
exclude = dfXA.node_id[np.isin(dfXA.index, ["ERR5308556", "ERR5414941", "ERR5651144"]) == False].values

arg.plot_pango_subgraph(
    "XA",
    height=500,
    exclude=exclude,
    y_axis_scale="rank",
    oldest_y_label="2020-12",
    child_levels=0,
    parent_levels=2,
    positions_file="subgraph_layouts/XA-min.json",
    highlight_nodes={'yellow': dfXA.node_id},
)

In [ ]:
dfXN = df[np.logical_and(df.pango == "XN", df.is_sample)]
dfXAU = df[np.logical_and(df.pango == "XAU", df.is_sample)]

keepXN_XAU = ["SRR20786472", "ERR9626571", "ERR9627436", "ERR9827586"]
inclBA2 = df.loc[["ERR9502469", "ERR9794107", "SRR20777279", "ERR9794253", "ERR9969775"], 'node_id']

exclude = list(dfXN.node_id[np.isin(dfXN.index, keepXN_XAU) == False])
exclude += list(dfXAU.node_id[np.isin(dfXAU.index, keepXN_XAU) == False])

arg.plot_pango_subgraph(
    ["XN", "XAU"],
    height=500,
    exclude=exclude,
    include=inclBA2,
    y_axis_scale="time",
    oldest_y_label="2021-12",
    child_levels=0,
    parent_levels=2,
    positions_file="subgraph_layouts/XN-XAU-min.json",
    highlight_nodes={'purple': dfXN.node_id, 'cyan': dfXAU.node_id},
)

In [ ]:
colours = ['#332288', '#88CCEE', '#44AA99', '#999933', '#DDCC77']  # from https://personal.sron.nl/~pault/
pangoX = ["XZ", "XAC", "XAD", "XAE", "XAP"]

keep = [
    "SRR19523737", "SRR20775418", "SRR20505514",
    "ERR9216823", "SRR19560135", "SRR19710932", "SRR20019588",
    "SRR19710932", "SRR18434609", "SRR21342521", "ERR9762269", "SRR20572879", "SRR20568702",
    "SRR19495684", "ERR9761341", "SRR19672153",
]

keep_ids = list(df.loc[["SRR19689888", "ERR8146303", "ERR8163061", "SRR19689888"], 'node_id'])

exclude = []
cmap = {}
for c, pX in zip(colours, pangoX):
    df_tmp = df[np.logical_and(df.pango == pX, df.is_sample)]
    exclude += list(df_tmp.node_id[np.isin(df_tmp.index, keep) == False])
    cmap[c] = list(df_tmp.node_id)

arg.plot_pango_subgraph(
    pangoX,
    exclude=exclude,
    y_axis_scale="rank",
    height=600,
    include=keep_ids,
    parent_levels=4,
    child_levels=0,
    highlight_nodes=cmap,
    oldest_y_label="2021-12",
    positions_file="subgraph_layouts/XZ-XAC-XAD-XAE-XAP-min.json",
)

In [ ]:
# Example of finding XZ descendants of a node such as the most recent displayed XZ node (ID 1228738)
# You can get the node IDs by switching to displaying '#ID' in the visualizer toolbar above the subgraph plot
from collections import Counter

# look at all XZs
for local_root in (1228738, 1225255, 1163537):
    c = Counter(ts.node(u).metadata['pango'] for u in ts.first().samples(local_root) if u != local_root)
    print(f"Sample descendants of {local_root} = {dict(c)}")

So SRR19523737 (#1228738) has 6/6 XZ descendant samples, SRR20775418 (#1225255) has 4, and the origin of the XZ node (the dots leading to the number 29) represent 48 - 3 - 6 - 4 = 35 XZ descendant samples.

You sould be able to check on this by counting up all the XZ samples: they should come to 6 + 4 + 35, plus the 3 that are shown = 48

In [ ]:
sum(np.logical_and(df.pango == "XZ", df.is_sample))